### Kulak & Grondin Example 8.2
This example is from "*Limit States Design in Structural Steel*", Ninth Edition, CISC, by Kulak and Grondin.
This example is slightly modified in order to compare the results with 
[that obtained](../../slope-deflection/KG-Example-8.2.ipynb) using
the method of slope-deflection.  The modifications
to the model are:
* a pin is inserted at the top of columns JK and IG, with the result that column stacks JKL and IGH
offer no lateral resistance (in the original K&G example, the columns are continuous and there is a very
small horizontal reaction at the base of those two columns).
* the cross-sectional areas of all members are made very large so as to make axial
changes in length neglible.

In [1]:
from salib import showImage
import salib.nbloader
import Frame2D_v03 as f2d

![Frame](KG82.d/KG82a.jpg)
The *notional* lateral loads of 6.9 kN at node **H** and 8.4 kN at node **G**
are added as [before](example-KG82.ipynb).

In [2]:
frame = f2d.Frame2D()
frame.loadStructuralData('KG82sd')   # read the CSV files in directory 'KG82sd.d'

In [3]:
frame.doall()
frame.saveStructuralData('KG82sd')


Time to solve = 0.01 sec.

ndof = 36, nfree = 26, nconstraints = 10



Nodes:

Node          X         Y  Constraints  DOF #s
----      -----     -----  -----------  ------
A         10000         0  FX,FY,MZ     26,27,28
B         10000      6500               0,1,2
C         10000     12000               3,4,5
D         20500         0  FX,FY,MZ     29,30,31
E         20500      6500               6,7,8
F         20500     12000               9,10,11
G         30500      6500               12,13,14
H         30500     12000               15,16,17
I         30500         0  FX,FY        32,33,18
J             0         0  FX,FY        34,35,19
K             0      6500               20,21,22
L             0     12000               23,24,25



Members:

Member   Node-J  Node-K    Length       dcx       dcy  Size                Ix           A  Releases
------   ------  ------    ------   -------   -------  --------      --------       -----  --------
AB       A       B         6500.0  

### Compare with results from slope-deflection

In [4]:
import pandas as pd

BM = [('AB', -49.9080183705027, 47.4941396356060),   # results from method of slope deflection
     ('BC', 250.526060428376, 252.096857589821),
     ('DE', -189.782099213905, -232.254022051198),
     ('EF', -290.011616822927, -333.061301195269),
     ('BE', -298.020200063982, 522.265638874126),
     ('CF', -252.096857589821, 333.061301195269)]

SD = pd.DataFrame({m:{'MZJ':-a,'MZK':-b} for m,a,b in BM}).T
SD

,MZJ,MZK
AB,49.908018,-47.494140
BC,-250.526060,-252.096858
BE,298.020200,-522.265639
CF,252.096858,-333.061301
DE,189.782099,232.254022
EF,290.011617,333.061301


In [5]:
R = frame.get_mefs()       # get our member end forces for the same members
#R[['MZJ']] - SD[['MZJ']]
R = R.ix[SD.index]
R

,FXJ,FYJ,MZJ,FXK,FYK,MZK
AB,995932.392434,371.365815,4.990802e+07,-995932.392434,-371.365815,-4.749414e+07
BC,453539.100854,-91385.984914,-2.505261e+08,-453539.100854,91385.984914,-2.520969e+08
BE,-91757.350586,267393.291580,2.980202e+08,91757.350586,310106.708420,-5.222656e+08
CF,91385.983398,228539.100854,2.520969e+08,-91385.983398,243960.899146,-3.330613e+08
DE,1054067.607566,64928.635068,1.897821e+08,-1054067.607566,-64928.635068,2.322540e+08
EF,468960.899146,113285.983887,2.900116e+08,-468960.899146,-113285.983887,3.330613e+08


#### % Difference in End Moments

In [6]:
m = R[['MZJ','MZK']]*1E-6
pd = (100*(m - SD[['MZJ','MZK']])/m).round(6)
pd

,MZJ,MZK
AB,-0.000000,0.000001
BC,-0.000000,-0.000000
BE,-0.000000,-0.000000
CF,-0.000000,-0.000001
DE,0.000002,0.000001
EF,-0.000001,-0.000001


Max. difference is 0.0004%, which is quite good.

In [7]:
pd.abs().max()

MZJ    0.000002
MZK    0.000001
dtype: float64